# 라이브러리 호출

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import re

plt.rc('font', family='nanum')
sns.set(font="Malgun Gothic",#"NanumGothicCoding", 
        rc={"axes.unicode_minus":False}, # 마이너스 부호 깨짐 현상 해결
        style='darkgrid')   

from xgboost import XGBRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import VotingRegressor
from sklearn.neighbors import KNeighborsRegressor
import catboost as cbt


from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.inspection import permutation_importance

import DataAnalysis_ksw as da
import Util_for_sklearn_ksw as ufs


# 데이터 불러오기

In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('train.csv')

## 로그변환

## 데이터 분리

In [3]:
x = train.drop(['box_off_num'],axis=1)
y = train['box_off_num']

In [4]:
x_train, x_val, y_train, y_val = train_test_split(x,y,test_size=0.3)